In [1]:
# @title Import Libraries
import sys
import glob, os
import cv2
import numpy as np
import pandas as pd
import geopy.distance
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import exposure
from skimage import feature
import skimage.measure


2025-01-06 14:22:53.036656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-06 14:22:54.022368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2025-01-06 14:22:54.022466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/sit

In [2]:
# Setup paths and import local libraries
%cd ../../
%ls

prj_path = Path('SkyAI-Sim')
sys.path.insert(0, str(prj_path))
from src.utils import img_helper
from src.utils.io_helper import str_to_floats
from src.utils import consts
from src.data.skyai import SkyAI

args = consts.ARGS
args

/home/sdjkhosh/Projects/SkyAI-Sim
README.md  dataset/  requirements.txt  src/


usage: SkyAI Sim Project [-h] [--coords COORDS] [--fov FOV]
                         [--aspect_ratio ASPECT_RATIO [ASPECT_RATIO ...]]
                         [--utm UTM] [--map_type {satellite,roadmap,terrain}]
                         [--dataset {SkyAI,VBN}] [--data_dir DATA_DIR]
                         [--vmargin VMARGIN]
                         [--img_size IMG_SIZE [IMG_SIZE ...]]
                         [--overlap OVERLAP] [--batch_size BATCH_SIZE]
                         [--seed SEED]
SkyAI Sim Project: error: argument --fov: invalid float value: '/home/sdjkhosh/.local/share/jupyter/runtime/kernel-v35aad76d9681d819be17f09a29072c876f13a57c8.json'


Namespace(coords=[35.19, -89.94, 35.11, -89.8, 120.0], fov=78.8, aspect_ratio=[4, 3], utm='EPSG:32616', map_type='roadmap', dataset='SkyAI', data_dir='/home/sdjkhosh/Datasets/Memphis/', vmargin=20, img_size=[400, 400, 3], overlap=0.6, batch_size=8, seed=2024)

In [ ]:
aerial_data = SkyAI(
        args=args,
        map_type='roadmap',
        data_dir=args.data_dir,
        overlap=args.overlap
        )
# aerial_data.config(download_raster=False)
aerial_data

Namespace(coords=[35.19, -89.94, 35.11, -89.8, 120.0], fov=78.8, aspect_ratio=[4, 3], utm='EPSG:32616', map_type='roadmap', dataset='SkyAI', data_dir='/home/sdjkhosh/Datasets/Memphis/', vmargin=20, img_size=[400, 400, 3], overlap=0.6, batch_size=8, seed=2024)
Checking folder:
	 /home/sdjkhosh/Datasets/Memphis/roadmap_0.6 Folder Exists.


2025-01-06 14:23:01.205376: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-01-06 14:23:01.205436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: c2-parisa
2025-01-06 14:23:01.205453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: c2-parisa
2025-01-06 14:23:01.205584: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.54.0
2025-01-06 14:23:01.205635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.54.0
2025-01-06 14:23:01.205643: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.54.0
2025-01-06 14:23:01.206319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAP

In [ ]:
aerial_imgs_path = glob.glob(os.path.join(aerial_data.data_dir / 'images', "*.jpg"))
aerial_imgs_path.sort()
print(len(aerial_imgs_path))
aerial_imgs, aerial_titles = img_helper.choose_random_images(nx*ny,
                                                            aerial_imgs_path,
                                                            output_dir=aerial_data.data_dir / 'random_sample',
                                                            obj=aerial_data,
                                                            seed=seed)

In [ ]:
import skimage.measure
entropys = [skimage.measure.shannon_entropy(img) for img in aerial_imgs]

img_helper.plot_multy(aerial_imgs, aerial_data.data_dir /'entropies', nx, ny, entropys)

In [ ]:
road_n = [cv2.normalize(img, None, alpha = 0, beta = 255,
                       norm_type = cv2.NORM_MINMAX,
                       dtype = cv2.CV_32F).astype('uint8') for img in aerial_imgs]
road_gray = [cv2.cvtColor(real_img, cv2.COLOR_BGR2GRAY) for real_img in road_n]
img_helper.plot_multy(road_gray, '../Gray Scale \nRoadmap Images', nx, ny, aerial_titles)

In [ ]:

road_eq = [cv2.equalizeHist(img) for img in road_gray]
img_helper.plot_multy(road_eq, '../Equalized Gray Scale \nRoadmap Images', nx, ny, aerial_titles)

In [ ]:
# equalize each color channel
eq_color = [cv2.merge([cv2.equalizeHist(channel) for channel in cv2.split(img)]) for img in road_n]
img_helper.plot_multy(eq_color, '../Equalized Color \nRoadmap Images', nx, ny, aerial_titles)